In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import json
import random
import pandas as pd
import shutil

<IPython.core.display.Javascript object>

In [3]:
!mkdir prepare_data/obj_det_js_tf/train_imgs_annot
!mkdir prepare_data/obj_det_js_tf/valid_imgs_annot
!mkdir prepare_data/obj_det_js_tf/train_imgs_annot/images/
!mkdir prepare_data/obj_det_js_tf/valid_imgs_annot/images/

<IPython.core.display.Javascript object>

In [4]:
gt_df = pd.read_csv("prepare_data/obj_det_js_tf/all_images_gt/od_labels.csv",)
print(gt_df.shape)
gt_df.head()

(161, 8)


,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,apple,76,31,410,350,7385219.jpg,705,724
1,orange,169,172,396,465,7385219.jpg,705,724
2,apple,143,54,357,341,3503298.jpg,730,725
3,banana,101,186,481,363,3503298.jpg,730,725
4,apple,81,40,473,332,3503298.jpg,730,725


<IPython.core.display.Javascript object>

In [5]:
train_valid_split = 0.7
nimages = gt_df["image_name"].nunique()
ntrain = int(train_valid_split * nimages)
nvalid = nimages - ntrain
print(nimages, ntrain, nvalid)

100 70 30


<IPython.core.display.Javascript object>

In [6]:
labels = ["apple", "orange", "banana"]
labels_map = {k: v for v, k in enumerate(labels)}
with open("prepare_data/obj_det_js_tf/labels_map.json", "w") as fp:
    json.dump(labels_map, fp)
labels_map

{'apple': 0, 'orange': 1, 'banana': 2}

<IPython.core.display.Javascript object>

In [7]:
gt_df["class_id"] = gt_df["label_name"].map(labels_map)
gt_df["image_id"] = gt_df["image_name"].rank(method="dense").astype(int)
gt_df.head()

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height,class_id,image_id
0,apple,76,31,410,350,7385219.jpg,705,724,0,67
1,orange,169,172,396,465,7385219.jpg,705,724,1,67
2,apple,143,54,357,341,3503298.jpg,730,725,0,24
3,banana,101,186,481,363,3503298.jpg,730,725,2,24
4,apple,81,40,473,332,3503298.jpg,730,725,0,24


<IPython.core.display.Javascript object>

In [8]:
images = gt_df["image_name"].unique().tolist()
train_images = random.sample(images, ntrain)
valid_images = list(set(images) - set(train_images))
print(len(images), len(train_images), len(valid_images))

100 70 30


<IPython.core.display.Javascript object>

In [9]:
gt_df_train = gt_df[gt_df["image_name"].isin(train_images)]
gt_df_valid = gt_df[gt_df["image_name"].isin(valid_images)]
print(len(gt_df_train), len(gt_df_valid))

117 44


<IPython.core.display.Javascript object>

In [10]:
# "images": [
#     {
#       "file_name": "34692938506_efda47f39a_n.jpg",
#       "height": 149,
#       "width": 240,
#       "id": 36
#     },
#     {
#       "file_name": "34696730346_5f0c131e59_n.jpg",
#       "height": 159,
#       "width": 240,
#       "id": 37
#     },

<IPython.core.display.Javascript object>

In [11]:
def create_images_dict(dft):
    images = dft["image_name"].unique().tolist()
    json_images = []
    for image in images:
        image_row_dct = dft[dft["image_name"] == image].iloc[0].to_dict()
        image_dct = {
            "file_name": image,
            "height": image_row_dct["image_height"],
            "width": image_row_dct["image_width"],
            "id": image_row_dct["image_id"],
        }
        json_images.append(image_dct)

    return json_images

<IPython.core.display.Javascript object>

In [12]:
# "annotations": [
#     { "image_id": 36, "bbox": [41, 3, 128, 122], "category_id": 2 },
#     { "image_id": 37, "bbox": [35, 39, 93, 76], "category_id": 1 },

<IPython.core.display.Javascript object>

In [13]:
def create_annots_dict(dft):
    json_annotations = []
    for indx, obj in dft.iterrows():
        annotations = {
            "image_id": obj["image_id"],
            "bbox": [
                obj["bbox_x"],
                obj["bbox_y"],
                obj["bbox_width"],
                obj["bbox_height"],
            ],
            "category_id": obj["class_id"],
        }
        json_annotations.append(annotations)

    return json_annotations

<IPython.core.display.Javascript object>

In [14]:
json_images = create_images_dict(gt_df_train)
json_annotations = create_annots_dict(gt_df_train)

json_train = {"images": json_images, "annotations": json_annotations}

with open("prepare_data/obj_det_js_tf/train_imgs_annot/annotations.json", "w") as fp:
    json.dump(json_train, fp)

<IPython.core.display.Javascript object>

In [15]:
json_images = create_images_dict(gt_df_valid)
json_annotations = create_annots_dict(gt_df_valid)

json_valid = {"images": json_images, "annotations": json_annotations}

with open("prepare_data/obj_det_js_tf/valid_imgs_annot/annotations.json", "w") as fp:
    json.dump(json_valid, fp)

<IPython.core.display.Javascript object>

In [16]:
for image in gt_df_train["image_name"].values:
    shutil.copy(
        "prepare_data/obj_det_js_tf/all_images_gt/" + image,
        "prepare_data/obj_det_js_tf/train_imgs_annot/images/",
    )

<IPython.core.display.Javascript object>

In [17]:
for image in gt_df_valid["image_name"].values:
    shutil.copy(
        "prepare_data/obj_det_js_tf/all_images_gt/" + image,
        "prepare_data/obj_det_js_tf/valid_imgs_annot/images/",
    )

<IPython.core.display.Javascript object>

In [18]:
print(json.dumps(json_valid, indent=4))

{
    "images": [
        {
            "file_name": "5095679.jpg",
            "height": 659,
            "width": 606,
            "id": 38
        },
        {
            "file_name": "9138249.jpg",
            "height": 709,
            "width": 605,
            "id": 95
        },
        {
            "file_name": "9963312.jpg",
            "height": 762,
            "width": 673,
            "id": 100
        },
        {
            "file_name": "7737552.jpg",
            "height": 727,
            "width": 629,
            "id": 73
        },
        {
            "file_name": "7813305.jpg",
            "height": 724,
            "width": 687,
            "id": 75
        },
        {
            "file_name": "4567289.jpg",
            "height": 647,
            "width": 642,
            "id": 31
        },
        {
            "file_name": "9490322.jpg",
            "height": 677,
            "width": 616,
            "id": 98
        },
        {
            "file_name": "

<IPython.core.display.Javascript object>